In [1]:
import requests
from bs4 import BeautifulSoup
def get_all_pages():
    urls = []
    page_number = 1
    
    for i in range(3):
        i = f"https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p={page_number}"
        page_number += 1   
        urls.append(i)
    return urls
urls = get_all_pages()
urls

['https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p=1',
 'https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p=2',
 'https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p=3']

In [23]:
import requests
from bs4 import BeautifulSoup

def get_recipe():
    base_urls = [
        "https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p=",  
        "https://www.marmiton.org/recettes/selection_sans_gluten.aspx?p=",
        "https://www.marmiton.org/recettes/selection_vegetarien.aspx",
        "https://www.marmiton.org/recettes/selection_mincealors.aspx"
    ]
    recipe_links = []

    for base_url in base_urls:
        for page_number in range(1, 4):  # Scrappe les 2 premières pages pour chaque base_url
            url = base_url + str(page_number)
            response = requests.get(url)
            soup = BeautifulSoup(response.content, "html.parser")
            print(f"on scrappe la page {page_number} avec l'url {url}")
            for card in soup.find_all('div', class_='recipe-card'):
                link = card.find('a', class_='recipe-card-link', href=True)
                if link:
                    recipe_links.append(link['href'])

    return recipe_links

recipe_links = get_recipe()
print(len(recipe_links))  # Affiche le nombre total de recettes collectées


on scrappe la page 1 avec l'url https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p=1
on scrappe la page 2 avec l'url https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p=2
on scrappe la page 3 avec l'url https://www.marmiton.org/recettes/selection_recette_vegan.aspx?p=3
on scrappe la page 1 avec l'url https://www.marmiton.org/recettes/selection_sans_gluten.aspx?p=1
on scrappe la page 2 avec l'url https://www.marmiton.org/recettes/selection_sans_gluten.aspx?p=2
on scrappe la page 3 avec l'url https://www.marmiton.org/recettes/selection_sans_gluten.aspx?p=3
on scrappe la page 1 avec l'url https://www.marmiton.org/recettes/selection_vegetarien.aspx1
on scrappe la page 2 avec l'url https://www.marmiton.org/recettes/selection_vegetarien.aspx2
on scrappe la page 3 avec l'url https://www.marmiton.org/recettes/selection_vegetarien.aspx3
on scrappe la page 1 avec l'url https://www.marmiton.org/recettes/selection_mincealors.aspx1
on scrappe la page 2 avec l'url https://

In [20]:
def scrape_recipe(recipe_url):
    response = requests.get(recipe_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraire le titre de la recette
    title = soup.find('div', class_='main-title').find('h1').text.strip() if soup.find('div', class_='main-title') else "Titre non trouvé"

    
    # Extraire le temps de préparation
    prep_time = soup.find('div', class_='recipe-primary__item').find('span').text.strip() if soup.find('div', class_='recipe-primary__item') else "Temps de préparation non trouvé"
       
    # Temps de repos
    time_details = soup.find('div', class_='time__details')  
    if time_details:
        time_divs = time_details.find_all('div') 
        if len(time_divs) >= 5: 
            rest_time = time_divs[3].text.strip()  
        else:
            rest_time = "Temps de repos non trouvé"
    else:
        rest_time = "Temps de repos non trouvé"
    
    time_details = soup.find('div', class_='time__details')  
    if time_details:
        time_spans = time_details.find_all('div')  
        if len(time_spans) >= 5:
            cook_time = time_spans[5].text.strip() 
        else:
            cook_time = "Temps de cuisson non trouvé"
    else:
        cook_time = "Temps de cuisson non trouvé"

    ingredients = []
    
    ingredient_sections = soup.find_all('span', class_='card-ingredient-title') 
    
    for section in ingredient_sections:
        # Nom de l'ingrédient
        name = section.find('span', class_='ingredient-name')
        name = name.text.strip() if name else "Nom non trouvé"
        
        # Quantité
        quantity = section.find('span', class_='count')
        quantity = quantity.text.strip() if quantity else "Quantité non trouvée"
        
        # Unité 
        unit = section.find('span', class_='unit')
        unit = unit.text.strip() if unit else "Unité non trouvée"  
        
        # Complément
        complement = section.find('span', class_='ingredient-complement')
        complement = complement.text.strip() if complement else "Complément non trouvé"
        
        ingredients.append({
            'name': name,
            'quantity': quantity,
            'unit': unit,  
            'complement': complement
        })

    steps = []

    step_containers = soup.find_all('div', class_='recipe-step-list__container')
    
    for container in step_containers:
        p_tags = container.find_all('p')  
        for tag in p_tags:
            steps.append(tag.text.strip())
        
       # Extraire l'image
    image_tag = soup.find('div', class_='recipe-media-viewer-media-container recipe-media-viewer-media-container-picture-only')
    if image_tag:
        image_tag = image_tag.find('img') 
        if image_tag:

            image_url = image_tag.get('data-src') or image_tag.get('src')
        else:
            image_url = "Image non trouvée"
    else:
        image_url = "Image non trouvée"

    return {
        'title': title,
        'prep_time': prep_time,
        'repos': rest_time,
        'cuisson': cook_time,
        'ingredients': ingredients,
        'steps': steps,
        'image_url': image_url
    }

all_recipes = []
for recipe_link in recipe_links:
    print(f"Scraping: {recipe_link}")
    recipe_data = scrape_recipe(recipe_link)
    all_recipes.append(recipe_data)



Scraping: https://www.marmiton.org/recettes/recette_falafel-croquettes-de-pois-chiches_23038.aspx
Scraping: https://www.marmiton.org/recettes/recette_salade-de-quinoa_29581.aspx
Scraping: https://www.marmiton.org/recettes/recette_samossas-de-legumes_46288.aspx
Scraping: https://www.marmiton.org/recettes/recette_frita_13781.aspx
Scraping: https://www.marmiton.org/recettes/recette_salade-de-concombres-et-melons-a-la-menthe_31646.aspx
Scraping: https://www.marmiton.org/recettes/recette_salade-de-lentilles-pour-l-ete_170891.aspx
Scraping: https://www.marmiton.org/recettes/recette_salade-aigre-douce-aux-concombres-thailande_23848.aspx
Scraping: https://www.marmiton.org/recettes/recette_coleslaw-de-chou-rouge-carotte-et-pomme_529825.aspx
Scraping: https://www.marmiton.org/recettes/recette_empanadas-au-tofu_67200.aspx
Scraping: https://www.marmiton.org/recettes/recette_roulades-d-aubergine-vegan_345913.aspx
Scraping: https://www.marmiton.org/recettes/recette_salade-toute-orange_308774.aspx
Sc

In [22]:
for recipe in all_recipes :
    print(recipe)  # Affiche toutes les recettes
    print("\n\n")  # Ajoute une ligne vide entre chaque recette

{'title': 'Falafel (croquettes de pois chiches)', 'prep_time': '1h20', 'repos': '-', 'cuisson': '20 min', 'ingredients': [{'name': 'huile de friture', 'quantity': '', 'unit': '', 'complement': ''}, {'name': 'sel', 'quantity': '', 'unit': '', 'complement': ''}, {'name': 'coriandre', 'quantity': '1', 'unit': 'cuillère à café', 'complement': 'en poudre'}, {'name': 'cumin en poudre', 'quantity': '1', 'unit': 'cuillère à café', 'complement': ''}, {'name': 'farine', 'quantity': '3', 'unit': 'cuillères à soupe', 'complement': ''}, {'name': 'persil', 'quantity': '1', 'unit': 'bouquet', 'complement': ''}, {'name': 'oignon', 'quantity': '1', 'unit': '', 'complement': 'moyen'}, {'name': 'pois chiches', 'quantity': '200', 'unit': 'g', 'complement': ''}, {'name': 'fèves', 'quantity': '500', 'unit': 'g', 'complement': 'sèches'}, {'name': 'ail', 'quantity': '2', 'unit': 'gousses', 'complement': ''}, {'name': 'paprika', 'quantity': '1', 'unit': 'cuillère à café', 'complement': ''}, {'name': 'basilic f